## Preparing Interface Data

In [1]:
import json as json
import pandas as pd

- Upload CVD Cell2PMID data and OS Cell2PMID data

In [2]:
with open("../../caseolap-cvd/data/textcube_cell2pmid.json",'r') as f1:
    texcube_cell2pmid_cvd = json.load(f1)

In [3]:
with open("../../caseolap-os/data/textcube_cell2pmid.json",'r') as f1:
    texcube_cell2pmid_os = json.load(f1)


- CVD and OS catagories

In [4]:
CVD = ["CM","ARR","CHD","VD","IHD","CCS","VOO","OHD"]
OS = ["IOS", "ROS","OOS"]

#### Intersection of Documents

- Find the intersection of the documents among the CVDs and OSs

In [5]:
DATA = []

for os,cell2pmid_os in zip(OS,texcube_cell2pmid_os):
    data = {"OS":os}
    
    for cvd,cell2pmid_cvd in zip(CVD,texcube_cell2pmid_cvd):
        
        '''Find the intersection of two sets'''
        ints = set(cell2pmid_os).intersection(set(cell2pmid_cvd))
        
        data.update({cvd:len(ints)})
        
    DATA.append(data)
    #print(os,cvd,len(ints))

In [6]:
#DATA

In [7]:
df = pd.DataFrame(DATA)
df

,OS,CM,ARR,CHD,VD,IHD,CCS,VOO,OHD
0,IOS,4900,1565,1289,423,10388,332,126,1957
1,ROS,26351,12903,5296,3243,39348,5638,738,14204
2,OOS,1073,216,103,136,1633,75,28,344


In [8]:
df.to_csv("cvd-os-ints-docs.csv")

### Intersection of Proteins

In [24]:
data_cvd = pd.read_csv("../../caseolap-cvd/result/caseolap.csv")
data_cvd = data_cvd.set_index("protein")
data_cvd.head()

,CM,ARR,CHD,VD,IHD,CCS,VOO,OHD
protein,,,,,,,,
P05556,0.018217,0.007162,0.008317,0.000000,0.035930,0.008059,0.0000,0.023034
O94759,0.025501,0.021015,0.000000,0.000000,0.024629,0.000000,0.0000,0.021674
Q14957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.007557
P14416,0.032675,0.036637,0.008104,0.000000,0.020849,0.029931,0.0000,0.020861
Q5R352,0.145437,0.070132,0.012846,0.038515,0.081818,0.029315,0.0375,0.146208


In [25]:
data_os = pd.read_csv("../../caseolap-os/result/caseolap.csv")
data_os = data_os.set_index("protein")
data_os.head()

,IOS,ROS,OOS
protein,,,
Q5VVH1,0.000000,0.031676,0.000000
Q5S007,0.000000,0.027231,0.000000
P17252,0.180848,0.326200,0.038242
Q13023,0.000000,0.172168,0.000000
O75052,0.000000,0.075413,0.000000


In [11]:
CVD = ["CM","ARR","CHD","VD","IHD","CCS","VOO","OHD"]
OS = ["IOS", "ROS","OOS"]

In [28]:
data_cvd.loc["P05556"]["CM"]

0.018216646028645818

In [31]:
DATA = []

for os in OS:
    data = {"OS":os}
    
    os_name = data_os.index
    d_os = data_os[os]
    
    scored_name_os = []
    for osname,os_score in zip(os_name,d_os):
        if os_score > 0:
            scored_name_os.append(osname)
    
    for cvd in CVD:
        
        total_score = 0
        
        d_cvd = data_cvd[cvd]
        cvd_name = data_cvd.index
        
        
        scored_name_cvd = []
        for cvdname,cvd_score in zip(cvd_name,d_cvd):
            if cvd_score > 0:
                scored_name_cvd.append(cvdname)
            
                
        ints = set(scored_name_os).intersection(set(scored_name_cvd))
        
        
        for p in list(ints):
            cvd_score = data_cvd.loc[p][cvd]
            os_score = data_os.loc[p][os]
            total_score += cvd_score+os_score
            
        
        
        data.update({cvd: total_score})
        print(os,cvd,len(ints),total_score)
        
    DATA.append(data)

IOS CM 65 9.999243459013247
IOS ARR 55 8.401151329934844
IOS CHD 37 5.42166371768585
IOS VD 45 6.53287183268733
IOS IHD 61 9.30628183584397
IOS CCS 49 7.224206654182383
IOS VOO 26 4.413471835679125
IOS OHD 64 9.609320665873916
ROS CM 94 22.738614161442342
ROS ARR 73 19.11694707385757
ROS CHD 43 13.20887002038874
ROS VD 58 15.478819098017482
ROS IHD 77 20.421802749260397
ROS CCS 63 16.893568888223584
ROS VOO 28 8.806732460488478
ROS OHD 91 21.949738650501242
OOS CM 44 6.334817312529893
OOS ARR 36 4.981727403286238
OOS CHD 25 3.623073772066045
OOS VD 33 4.221772765247149
OOS IHD 43 5.8860542939995595
OOS CCS 33 4.208642246166234
OOS VOO 23 3.166420710284887
OOS OHD 45 5.948885547040305


In [32]:
df2 = pd.DataFrame(DATA)
df2

,OS,CM,ARR,CHD,VD,IHD,CCS,VOO,OHD
0,IOS,9.999243,8.401151,5.421664,6.532872,9.306282,7.224207,4.413472,9.609321
1,ROS,22.738614,19.116947,13.208870,15.478819,20.421803,16.893569,8.806732,21.949739
2,OOS,6.334817,4.981727,3.623074,4.221773,5.886054,4.208642,3.166421,5.948886


In [33]:
df2.to_csv("cvd-os-ints-prot.csv")

### Merged Data

**1. Import Ion-channel data prepared from implementation of Uniprot API**

In [116]:
with open("../../UniprotAPI/proteins.json",'r') as pf:
    Proteins = json.load(pf)

**2. Prepare a dictionary with Uniprot ``ID`` as key and value as a another dictionary with ``name`` (recomended name) and ``synonyms`` keys and value.**

In [117]:
DATA = {}
for item in Proteins:
    ID = item['ID']
    names = item['names']
    DATA.update({ID:{"name":names[0],"synonyms": names}})

**3. Update CVD scores **

In [118]:
CVD = ["CM","ARR","CHD","VD","IHD","CCS","VOO","OHD"]
OS = ["IOS", "ROS","OOS"]

- 3.1: Set 0.0 score by defult

In [119]:
for key,value in DATA.items():
    for cvd in CVD:
        DATA[key].update({cvd:0.0})
    for os in OS:
        DATA[key].update({os:0.0})

- 3.2 Update CVD data

In [120]:
"""Load CVD data"""
data_cvd = pd.read_csv("../../caseolap-cvd/result/caseolap.csv")
data_cvd.head()

,protein,CM,ARR,CHD,VD,IHD,CCS,VOO,OHD
0,Q9UKS6,0.000000,0.000000,0.000000,0.000000,0.011396,0.000000,0.000000,0.000000
1,P17612,0.020392,0.007317,0.008494,0.008650,0.016795,0.008234,0.000000,0.015174
2,P0DP23,0.033334,0.114456,0.015841,0.014470,0.027834,0.086746,0.007252,0.032223
3,Q6TME0,0.016664,0.000000,0.000000,0.017726,0.000000,0.000000,0.000000,0.007519
4,Q96D31,0.000000,0.007391,0.000000,0.000000,0.000000,0.000000,0.000000,0.007553


- 3.3  set protein as index

In [121]:
data_cvd = data_cvd.set_index("protein")
data_cvd.head()

,CM,ARR,CHD,VD,IHD,CCS,VOO,OHD
protein,,,,,,,,
Q9UKS6,0.000000,0.000000,0.000000,0.000000,0.011396,0.000000,0.000000,0.000000
P17612,0.020392,0.007317,0.008494,0.008650,0.016795,0.008234,0.000000,0.015174
P0DP23,0.033334,0.114456,0.015841,0.014470,0.027834,0.086746,0.007252,0.032223
Q6TME0,0.016664,0.000000,0.000000,0.017726,0.000000,0.000000,0.000000,0.007519
Q96D31,0.000000,0.007391,0.000000,0.000000,0.000000,0.000000,0.000000,0.007553


- 3.4 Reading a specific row data

In [122]:
idata = data_cvd.loc['P17612',:]
idata["CM"],idata["ARR"]

(0.020392203093457788, 0.007317158357731486)

- 3.5 Update CVD data in DATA dictionary

In [123]:
unscored = []
for key,value in DATA.items():
    try:
        idata = data_cvd.loc[key,:]
    except:
        unscored.append(key)
        
    for cvd in CVD:
        DATA[key].update({cvd:idata[cvd]})   

**4. Update OS data **

- 4.1 load os data

In [124]:
data_os = pd.read_csv("../../caseolap-os/result/caseolap.csv")
data_os.head()

,protein,IOS,ROS,OOS
0,Q12797,0.000000,0.056648,0.0
1,Q08499,0.000000,0.035435,0.0
2,P56539,0.120955,0.293942,0.0
3,P05556,0.034114,0.222639,0.0
4,Q14571,0.015186,0.114933,0.0


- 4.2 set protein as index

In [125]:
data_os = data_os.set_index("protein")
data_os.head()

,IOS,ROS,OOS
protein,,,
Q12797,0.000000,0.056648,0.0
Q08499,0.000000,0.035435,0.0
P56539,0.120955,0.293942,0.0
P05556,0.034114,0.222639,0.0
Q14571,0.015186,0.114933,0.0


- Update OS data in DATA dictionary

In [126]:
unscored = []
for key,value in DATA.items():
    try:
        idata = data_os.loc[key,:]
    except:
        unscored.append(key)
        
    for os in OS:
        DATA[key].update({os:idata[os]})   

In [127]:
#DATA

-  Save the data

In [128]:
with open("merged_protein_data.json","w") as fpp:
    json.dump(DATA,fpp)

In [131]:
DATAFRAME = []

for key,value in DATA.items():
    data = {}
    data.update({"ID": key})
    data.update(value)
    DATAFRAME.append(data)

df = pd.DataFrame(DATAFRAME)
df = df.set_index("ID")
df.head()

,name,synonyms,CM,ARR,CHD,VD,IHD,CCS,VOO,OHD,IOS,ROS,OOS
ID,,,,,,,,,,,,,
P17302,Gap junction alpha-1 protein,"[Gap junction alpha-1 protein, Connexin-43, Cx...",0.108147,0.154496,0.089738,0.034155,0.100026,0.098568,0.04216,0.103816,0.179578,0.349316,0.165879
Q13936,Voltage-dependent L-type calcium channel subun...,[Voltage-dependent L-type calcium channel subu...,0.011261,0.011770,0.000000,0.008714,0.000000,0.008294,0.00000,0.007532,0.000000,0.031717,0.000000
O95180,Voltage-dependent T-type calcium channel subun...,[Voltage-dependent T-type calcium channel subu...,0.011261,0.011770,0.000000,0.008714,0.000000,0.008294,0.00000,0.007532,0.000000,0.031717,0.000000
Q9NS61,Kv channel-interacting protein 2,"[Kv channel-interacting protein 2, KChIP2, A-t...",0.077975,0.123411,0.018526,0.023734,0.035045,0.058555,0.03109,0.068609,0.036451,0.278596,0.000000
Q8TD43,Transient receptor potential cation channel su...,[Transient receptor potential cation channel s...,0.018094,0.030867,0.000000,0.000000,0.018353,0.031019,0.00000,0.017291,0.030212,0.117727,0.000000


In [132]:
df.to_csv("merged-proteins.csv")